In [1]:
import pysr
import pandas as pd
import os
from pysr import PySRRegressor
import sympy
import numpy as np
import itertools
import random
import ast

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [ ]:
# create dataframes 
dir = "datasets/Feynman_with_units"
df_dict={}
for file in os.listdir(dir):
    fname = os.fsdecode(file)
    df = pd.read_csv(os.path.join(dir,fname),sep=" ",header=None)
    dfname,_ = os.path.splitext(fname)
    df_dict[dfname] = df
# now we instantiate pysr

print("DFs created")
file="best_equations.csv",  
best_list=[]
c=0


In [ ]:
# main loop
for name,df in df_dict.items():
    c+=1
    X=df.iloc[:10000,:-1]
    y=df.iloc[:10000,-1] 
    print(f"Going through DF {c} solving for equation {name}, X shape: {X.shape}, y shape: {y.shape}")
    model=PySRRegressor(niterations=5, warm_start=True, guesses=[],batching=True,batch_size=32,verbosity=0,unary_operators=["sqrt","exp","sin","cos"])
    model.fit(X,y)
    llm_called = random.random()
    while True:
        if llm_called<0.1: # 10% of the time
            model.guesses = 
            
            
                
        
        best=model.equations_.sort_values("score",axis=0,ascending=False).iloc[[0]] # score chooses the equation with the best complexity-accuracy trade off.
        best["name"] = name 
        best_list.append(best)
    
# now concat vertically and save
pd.concat(best_list,axis=0).to_csv('best_eqs.csv',index=False,header=["complexity","loss","equation","score","sympy format","lambda_format","Filename"])
 

In [12]:
symbols = sympy.symbols("""
sigma theta1 x1 x2 y1 y2 m1 m2 G z1
z2 m_0 v c x3 y3 mu Nn q1 q2
epsilon r Ef q B m u w r1 r2
z k_spring x t F omega omega_0 n theta2 d1
d2 Int_0 lambd d a p h I1 I2 delta
pr gamma kb n_0 mu_drift Volt mob V1 V2 rho
alpha kappa T1 T2 Pwr p_d y sigma_den chi n_rho
I rho_c_0 mom g_ Jz E_n Bx By Bz k
I_0 beta A_vec x0 x4 x5 x6 x7 x8 x9
""")
(sigma, theta1, x1, x2, y1, y2, m1, m2, G, z1,
z2, m_0, v, c, x3, y3, mu, Nn, q1, q2,
epsilon, r, Ef, q, B, m, u, w, r1, r2,
z, k_spring, x, t, F, omega, omega_0, n, theta2, d1,
d2, Int_0, lambd, d, a, p, h, I1, I2, delta,
pr, gamma, kb, n_0, mu_drift, Volt, mob, V1, V2, rho,
alpha, kappa, T1, T2, Pwr, p_d, y, sigma_den, chi, n_rho,
I, rho_c_0, mom, g_, Jz, E_n, Bx, By, Bz, k,
I_0, beta, A_vec,x0,x4,x5,x6,x7,x8,x9) = symbols

locals_ = {s.name: s for s in symbols}


In [13]:
# creating the core DF 

df_pred=pd.read_csv("best_eqs.csv")
df_original = pd.read_csv("~/projects/FeynmanEquations.csv")[["Filename","Formula"]]
df_merged = df_pred.merge(df_original,on="Filename",how='left')
df_merged["Equation"] = df_merged["Formula"].map(lambda f: sympy.sympify(f, locals=locals_))
df_merged["sympy format"] = df_merged["sympy format"].map(lambda f: sympy.sympify(f, locals=locals_))
df_merged= df_merged[["Filename","Equation","loss","sympy format"]]
df_merged.set_index("Filename")
df_merged.drop(df_merged[df_merged['Equation']==sympy.nan].index,inplace=True)




In [14]:
df_merged

,Filename,Equation,loss,sympy format
0,III.15.27,2*pi*alpha/(d*n),8.327730e-14,6.2831855*x0/(x1*x2)
1,III.4.33,h*omega/(2*pi*(exp(h*omega/(2*pi*T*kb)) - 1)),6.020358e-01,x2*x3
2,I.24.6,m*x**2*(omega**2 + omega_0**2)/4,6.909887e+00,x0*x3**2*(x2 - cos(x1))
3,III.9.52,8*pi*Ef*p_d*sin(t*(omega - omega_0)/2)**2/(h*t...,2.744946e+01,4.81302122383196*x0*x1*exp(cos(x4 - x5))/x3
4,I.34.14,omega_0*(1 + v/c)/sqrt(1 - v**2/c**2),1.010983e-02,x2*exp(x1/x0)
...,...,...,...,...
95,II.10.9,sigma_den/(epsilon*(chi + 1)),7.702505e-16,x0/(x1*(x2 + 1.0))
96,III.12.43,h*n/(2*pi),1.330285e-14,0.15915494*x0*x1
97,II.27.16,Ef**2*c*epsilon,1.120711e-10,x0*x1*x2**2
98,I.47.23,sqrt(gamma*pr/rho),7.385825e-15,sqrt(x0*x1/x2)


In [15]:
# i want to know how similar these are...
# constants are an issue.. we will try to prove equivalence up to a constant 
pred_eqs = df_merged['sympy format']
orig_eqs = df_merged['Equation']


In [ ]:
def check_equality(pred_expr, true_expr, true_varnames=None):
    '''
    used to check if two sympy expressions are "equal".
    equality is defined here as equal upto a constant.
    Works by sampling the true equation and then trying out permutations 
    of the input variables of the candidate solution and obtaining the ratio of the fits. 
    If the ratios are identical, the expressions are equivalent upto a constant.  
    
    '''
    # print("Free symbols in the true expression: ")
    # print(true_expr.free_symbols)
    pred_vars = sorted(pred_expr.free_symbols, key=lambda s: s.name)
    if true_varnames:
        true_vars = [sympy.Symbol(n) for n in true_varnames]
    else:
        true_vars = sorted(true_expr.free_symbols, key=lambda s: s.name)
    # print("variables in the true expression: ")
    print(true_vars)
    if len(pred_vars) != len(true_vars):
        return False
# lambdify
    f_pred = sympy.lambdify(pred_vars, pred_expr, "numpy")
    f_true = sympy.lambdify(true_vars, true_expr, "numpy")
    
# 10 points 
    N_samples = 10
    X = np.random.uniform(1.0, 5.0, (N_samples, len(true_vars)))
    
    # Calculate True Output
    try:
        y_true = f_true(*X.T) # evaluate true function over 10 points. 
    except Exception:
        return False 

    best_perm = None
    
    for perm_indices in itertools.permutations(range(len(true_vars))): # [0,1,2] eg if len is 3
        print(perm_indices) # [0,1,2], [0,2,1]
        try:
            # Reorder columns of X to match this permutation
            X_permuted = X[:, perm_indices]
            
            # Eval prediction
            y_pred = f_pred(*X_permuted.T) # get the fit on y after re-ordering. 
            
            # Calculate Ratio: y_pred / y_true
            # If equations are equivalent (up to constant), ratio should be identical for all points
            ratio = y_pred / y_true            
            # Check if ratio is constant (sigma is almost 0)
            if np.std(ratio) < 1e-5 and np.isfinite(ratio).all():
                # We found a candidate permutation! 
                # Map the indices back to symbols
                best_perm = [true_vars[i] for i in perm_indices]
                break
                
        except Exception:
            continue

    if best_perm:
        mapper = dict(zip(pred_vars, best_perm)) # make the correct mapping.
        pred_mapped = pred_expr.subs(mapper)
        
        # run simplify for a final sanity check 
        final_ratio = sympy.simplify(pred_mapped / true_expr)
        
        if final_ratio.is_constant():
            return True

    return False

In [ ]:
# finally, apply the above onto the dataframe and add the col
df_merged["equivalent"] = df_merged.apply(
    lambda row: check_equality(
        row["sympy format"],   # predicted
        row["Equation"]        # ground truth
    ),
    axis=1
)



In [18]:
df_merged[df_merged['equivalent']==True]


,Filename,Equation,loss,sympy format,equivalent
0,III.15.27,2*pi*alpha/(d*n),8.327730e-14,6.2831855*x0/(x1*x2),True
5,III.7.38,4*pi*B*mom/h,1.810693e-11,12.566371*x0*x1/x2,True
8,I.18.12,F*r*sin(theta),3.964229e-13,x0*x1*sin(x2),True
10,II.21.32,q/(4*pi*epsilon*r*(1 - v/c)),1.235921e-16,0.07957746*x0/(x1*x2*(-x3/x4 + 1.0)),True
11,II.6.15b,3*p_d*sin(theta)*cos(theta)/(4*pi*epsilon*r**3),2.287197e-17,0.23873241*x1*sin(x2)*cos(x2)/(x0*x3**3),True
12,II.34.29b,2*pi*B*Jz*g_*mom/h,7.030817e-10,6.2831855*x0*x2*x3*x4/x1,True
13,I.14.3,g*m*z,4.462443e-12,x0*x1*x2,True
14,I.13.12,G*m1*m2*(1/r2 - 1/r1),7.885416e-13,x0*x1*x4*(x2 - x3)/(x2*x3),True
16,I.12.5,Ef*q2,2.798131e-13,x0*x1,True
17,I.43.16,Volt*mu_drift*q/d,1.221977e-12,x0*x1*x2/x3,True


In [ ]:
df_merged.shape
# score (pysr with defaults) = 46/97 (appx 47%) # took at most a minute to run
# score (pysr with added sqrt, sin, exp, cos) = 55/97 (appx 56%) ~ took about 30 minutes to run

(97, 5)

In [26]:
import ast

tree = ast.parse('''import math
import numpy
def func(a,b): return a+b
''')
print(ast.dump(tree,indent=4))

tree.body[-1]
    

Module(
    body=[
        Import(
            names=[
                alias(name='math')]),
        Import(
            names=[
                alias(name='numpy')]),
        FunctionDef(
            name='func',
            args=arguments(
                posonlyargs=[],
                args=[
                    arg(arg='a'),
                    arg(arg='b')],
                kwonlyargs=[],
                kw_defaults=[],
                defaults=[]),
            body=[
                Return(
                    value=BinOp(
                        left=Name(id='a', ctx=Load()),
                        op=Add(),
                        right=Name(id='b', ctx=Load())))],
            decorator_list=[])],
    type_ignores=[])


In [2]:
from pydantic import BaseModel,Field,field_validator

toggle_strict_validation=True

class ResponseStructure(BaseModel):
    answer: str =  Field(...,min_length=0,max_length=200,)
    # must enforce string must be a python function 
    @field_validator("answer")
    @classmethod
    def check_output(cls,funcstr):
        # basic validation
        try:
            tree=ast.parse(funcstr)
        except SyntaxError as e:
            raise ValueError(f"Syntax error: {e}")        
        if toggle_strict_validation:
            if not isinstance(tree.body[-1],ast.FunctionDef):
                raise ValueError(f" {funcstr} no def statement ")
            
            func = tree.body[0] 
            
            # contain one return 
            if not isinstance(func.body[-1],ast.Return):
                raise ValueError(f"{funcstr} does not end with a return statement")
                    
            return funcstr
        return funcstr
                
        
schema = ResponseStructure.model_json_schema()

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:8080/v1", # 
    api_key = "sk-no-key-required"
)
MAX_ATTEMPTS = 20
attempts=0
parsed=None
out='{}'

correct_expr_list={}
wrong_expr_list=[]
while len(correct_expr_list)<=5:
    if attempts==MAX_ATTEMPTS:
        break  

    completion = client.chat.completions.create(
        model="local-model",
        messages=[
            {
                "role": "system",
                "content": "You are a physics expert."
            },
            {
                "role": "user",
                "content": f'''
                            Generate a valid Python function representing an influential 20th century physics equation. No comments. No escape 
                            characters. Follow the structure def func(args): return expression
                            DO NOT generate the equation for any of: 
                            {correct_expr_list.keys()}

                            '''
            
            }
        ],
        response_format={
            "type": "json_schema",
            "json_schema": {
                "name": "response",
                "schema": schema
            }
        },
        temperature=0.3,
        
    )
    
    try:
        out=completion.choices[0].message.content
        print(out)
        print("validating...")
        parsed = ResponseStructure.model_validate_json(out)
        print("validated successfully")
        eq_tree = ast.parse(parsed.answer)
        func_name = [node.name for node in ast.walk(eq_tree) if isinstance(node, ast.FunctionDef)][0] # there will only be one def
        correct_expr_list[func_name] = parsed.answer
        print("Correct Expr List: ", correct_expr_list)
    except Exception as e:
        wrong_expr_list.append(out)
        print("Wrong Expr List: ", wrong_expr_list)        
        print(f"Validation failed with {e}. Retrying. Attempt# {attempts+1}")
    attempts+=1

print(correct_expr_list)
    

# instead of generating all 5 at once maybe go one -> validate -> put it in the prompt for the next. 

{
    "answer": "def momentum_equation(mass, velocity):\n    return mass * velocity"
}

    
validating...
validated successfully
Correct Expr List:  {'momentum_equation': 'def momentum_equation(mass, velocity):\n    return mass * velocity'}
{
    "answer": "def einstein_equation(mass, velocity):\n    return mass * velocity ** 2"
}

    
validating...
validated successfully
Correct Expr List:  {'momentum_equation': 'def momentum_equation(mass, velocity):\n    return mass * velocity', 'einstein_equation': 'def einstein_equation(mass, velocity):\n    return mass * velocity ** 2'}
{
    "answer": "def special_relativity_equation(v, c):\n    return (1 / math.sqrt(1 - (v**2 / c**2))) * (1 / math.sqrt(1 - (v**2 / c**2)))"
}

    
validating...
validated successfully
Correct Expr List:  {'momentum_equation': 'def momentum_equation(mass, velocity):\n    return mass * velocity', 'einstein_equation': 'def einstein_equation(mass, velocity):\n    return mass * velocity ** 2', 'special_relativity_e

In [13]:
wrong_expr_list

[]

In [14]:
correct_expr_list

{'momentum_equation': 'def momentum_equation(mass, velocity):\n    return mass * velocity',
 'einstein_equation': 'def einstein_equation(mass, velocity):\n    return mass * velocity ** 2',
 'special_relativity_equation': 'def special_relativity_equation(v, c):\n    return (1 / math.sqrt(1 - (v**2 / c**2))) * (1 / math.sqrt(1 - (v**2 / c**2)))'}